In [ ]:
import transformers
import torch
import pandas as pd
from transformers import pipeline
import telebot

In [ ]:
from huggingface_hub import login

login("")

In [ ]:
# Initialize the Telegram bot
TELEGRAM_BOT_TOKEN = ''
bot = telebot.TeleBot(TELEGRAM_BOT_TOKEN)

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
text_generator = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

# Load the CSV file
file_path = 'Tauranga Container Terminal Cargo Schedule.csv'
data = pd.read_csv(file_path)
data['Vessel_Voyage'] = data['Vessel'] + ' ' + data['Outbound Voyage #'].astype(str)
relevant_data = data[['Vessel_Voyage', 'Current ETA', 'Tauranga Cut-Off']]

def get_eta(vessel_voyage):
    vessel_info = relevant_data[relevant_data['Vessel_Voyage'].str.contains(vessel_voyage, case=False, na=False)]
    if not vessel_info.empty:
        return vessel_info[['Vessel_Voyage', 'Current ETA', 'Tauranga Cut-Off']].to_string(index=False)
    else:
        return "Arrr, no such vessel be found!"

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Welcome! How can I assist you today? Use /eta <vessel_voyage> to get the ETA for a vessel.")

@bot.message_handler(commands=['eta'])
def send_eta(message):
    try:
        vessel_voyage = ' '.join(message.text.split()[1:]).strip()
        if vessel_voyage:
            eta_info = get_eta(vessel_voyage)
            messages = [
                {"role": "system", "content": "You are a Vessel planner, who provides current ETA of vessels and port Cut-off time for vessels.The format is YYYY-MM-DD HH:MM. Don't add apologues, additional information and recommendations"},
                {"role": "user", "content": f"What are the ETA and Cut-off for vessel(s): {vessel_voyage}?"},
                {"role": "system", "content": f"Here be the latest cargo schedule:\n{eta_info}"},
            ]

            response = text_generator(
                messages,
                max_new_tokens=128,
                eos_token_id=text_generator.tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.2,
                top_p=0.2,
            )[0]["generated_text"]

            # Extract the relevant part of the generated text
            if 'content' in response[-1]:
                bot.reply_to(message, response[-1]['content'])
            else:
                bot.reply_to(message, "Arrr, I couldn't generate a proper response!")

        else:
            bot.reply_to(message, "Please provide a vessel name. Usage: /eta <vessel_name>")
    except Exception as e:
        bot.reply_to(message, f"An error occurred: {str(e)}")

@bot.message_handler(func=lambda message: True)
def echo_all(message):
    bot.reply_to(message, message.text)

if __name__ == '__main__':
    bot.polling()